In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0225 13:51:55.567602 4411000256 deprecation.py:323] From <ipython-input-1-3c4f46c1a93b>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0225 13:51:55.568879 4411000256 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0225 13:51:55.569994 4411000256 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz


W0225 13:51:55.774053 4411000256 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0225 13:51:55.775898 4411000256 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0225 13:51:55.822123 4411000256 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.con

Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import os

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, plot_model
import keras.backend as K
from keras.datasets import mnist

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image

import numpy as np
np.random.seed(1)

import random
import math

Using TensorFlow backend.


In [3]:
(X_train, _), (_, _) = mnist.load_data()

X_train = X_train.reshape(-1, 28*28)/255.

In [49]:
x_dim = X_train.shape[1] # 784
z_dim = 100
h_dim = 128
batch_size = 128

x = Input(shape=(x_dim,))
D_h = Dense(h_dim)(x)
D_h = LeakyReLU(0.2)(D_h)
D_o = Dense(1, activation='sigmoid')(D_h)
DM = Model(x, D_o)

z = Input(shape=(z_dim,))
G_h = Dense(h_dim)(z)
G_h = LeakyReLU(0.2)(G_h)
G_o = Dense(x_dim, activation='sigmoid')(G_h)
GM = Model(z, G_o)

# del DM
# DM = load_model('DM.h5')
# del GM
# GM = load_model('GM.h5')

DM.trainable = True
DM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

g = GM(z)
DM.trainable = False
y = DM(g)
AM = Model(z, y)
AM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

GM.summary()
DM.summary()
AM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_56 (InputLayer)        (None, 100)               0         
_________________________________________________________________
dense_111 (Dense)            (None, 128)               12928     
_________________________________________________________________
leaky_re_lu_56 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_112 (Dense)            (None, 784)               101136    
Total params: 114,064
Trainable params: 114,064
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_55 (InputLayer)        (None, 784)               0         
_________________________________________________________________
dens

In [50]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m,n])

if not os.path.exists('out/'):
    os.makedirs('out/')

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
    
i=0
for it in range(1000000):
    r = np.random.randint(0, X_train.shape[0], batch_size)
    x_real = X_train[r]
    
    z_samples = sample_Z(batch_size, z_dim)
    x_fake = GM.predict(z_samples)
    
    D_loss_real, D_acc_real = DM.train_on_batch(x_real, real)
    D_loss_fake, D_acc_fake = DM.train_on_batch(x_fake, fake)
    
    D_loss = 0.5*(D_loss_real + D_loss_fake)
    D_acc = 0.5*(D_acc_real + D_acc_fake)
    
    z_samples = sample_Z(batch_size, z_dim)

    G_loss, G_acc = AM.train_on_batch(z_samples, real)
    
    if it%1000==0:
        DM.save('DM.h5')
        GM.save('GM.h5')
        
        samples = GM.predict(sample_Z(16, z_dim))
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i+=1
        plt.close(fig)
        
        print('iter:{}'.format(it))
        print('[D loss:{:.4}, acc:{:.4}] [G loss:{:.4}, acc:{:.4}]'.format(D_loss, D_acc, G_loss, G_acc))
        print()

iter:0
[D loss:1.732, acc:0.2109] [G loss:0.5435, acc:1.0]

iter:1000
[D loss:0.01118, acc:1.0] [G loss:5.975, acc:0.0]

iter:2000
[D loss:0.02708, acc:0.9961] [G loss:6.433, acc:0.0]

iter:3000
[D loss:0.07997, acc:0.9766] [G loss:4.62, acc:0.0]



KeyboardInterrupt: 